In [ ]:
import random

# Cache states
MODIFIED = "M"
EXCLUSIVE = "E"
SHARED = "S"
INVALID = "I"

class CacheLine:
    def __init__(self, tag, state=INVALID):
        self.tag = tag
        self.state = state

class Cache:
    def __init__(self, size=4):
        self.lines = [CacheLine(None, INVALID) for _ in range(size)]

    def find_line(self, tag):
        for line in self.lines:
            if line.tag == tag:
                return line
        return None

    def evict_lru(self, tag):
        evicted_line = random.choice(self.lines)
        evicted_line.tag = tag
        evicted_line.state = INVALID
        return evicted_line

class MemoryHierarchy:
    def __init__(self, num_processors=2, cache_size=4):
        self.processors = [Cache(cache_size) for _ in range(num_processors)]

    def access_memory(self, processor_id, tag):
        cache = self.processors[processor_id]
        line = cache.find_line(tag)

        if line is None:
            line = cache.evict_lru(tag)
            other_caches = [p for i, p in enumerate(self.processors) if i != processor_id]
            for other_cache in other_caches:
                other_line = other_cache.find_line(tag)
                if other_line is not None:
                    if other_line.state == MODIFIED:
                        line.state = SHARED
                        other_line.state = SHARED
                    elif other_line.state == EXCLUSIVE:
                        line.state = SHARED
                        other_line.state = SHARED
                    elif other_line.state == SHARED:
                        line.state = SHARED
                    break
            else:
                line.state = EXCLUSIVE

        return line

def main():
    hierarchy = MemoryHierarchy(num_processors=4, cache_size=4)
    num_accesses = 10
    tags = ["A", "B", "C", "D", "E", "F", "G", "H"]

    for i in range(num_accesses):
        processor_id = random.randint(0, len(hierarchy.processors) - 1)
        tag = random.choice(tags)
        print(f"Processor {processor_id} accesses memory address {tag}")
        line = hierarchy.access_memory(processor_id, tag)
        print(f"Cache line: tag={line.tag}, state={line.state}\n")

if __name__ == "__main__":
    main()


Processor 2 accesses memory address F
Cache line: tag=F, state=E

Processor 1 accesses memory address B
Cache line: tag=B, state=E

Processor 3 accesses memory address A
Cache line: tag=A, state=E

Processor 1 accesses memory address F
Cache line: tag=F, state=S

Processor 2 accesses memory address A
Cache line: tag=A, state=S

Processor 2 accesses memory address B
Cache line: tag=B, state=E

Processor 2 accesses memory address A
Cache line: tag=A, state=S

Processor 1 accesses memory address B
Cache line: tag=B, state=S

Processor 1 accesses memory address B
Cache line: tag=B, state=S

Processor 0 accesses memory address B
Cache line: tag=B, state=S



In [20]:
import random

class Block:
    def __init__(self, tag, data, state):
        self.tag = tag
        self.data = data
        self.state = state
        self.timestamp = 0



In [21]:

class Cache:
    def __init__(self, capacity, replacement_policy):
        self.capacity = capacity
        self.blocks = {}
        self.replacement_policy = replacement_policy

    def access(self, tag, data):
        if tag in self.blocks:
            block = self.blocks[tag]
            block.data = data
            block.timestamp = 0
            return True
        return False

    def insert(self, block):
        if len(self.blocks) < self.capacity:
            self.blocks[block.tag] = block
        else:
            victim = self.replacement_policy(self.blocks)
            del self.blocks[victim.tag]
            self.blocks[block.tag] = block

    def evict(self, tag):
        if tag in self.blocks:
            del self.blocks[tag]


In [22]:

def lru_policy(blocks):
    least_recently_used = min(blocks.values(), key=lambda b: b.timestamp)
    return least_recently_used


In [23]:

def main(memory_accesses, private_cache_capacity, llc_capacity):
    private_cache = Cache(private_cache_capacity, lru_policy)
    llc = Cache(llc_capacity, lru_policy)

    for access in memory_accesses:
        tag, data = access
        in_private_cache = private_cache.access(tag, data)
        in_llc = llc.access(tag, data)

        if not in_private_cache and not in_llc:
            block = Block(tag, data, "exclusive")
            llc.insert(block)
            private_cache.insert(block)

        for block in private_cache.blocks.values():
            block.timestamp += 1
        for block in llc.blocks.values():
            block.timestamp += 1

    return len(private_cache.blocks), len(llc.blocks)


In [24]:

# Example test case
memory_accesses = [(random.randint(1, 100), "data_" + str(i)) for i in range(200)]
private_cache_capacity = 10
llc_capacity = 50

private_cache_blocks, llc_blocks = main(memory_accesses, private_cache_capacity, llc_capacity)
print("Number of blocks in the private cache:", private_cache_blocks)
print("Number of blocks in the LLC:", llc_blocks)

Number of blocks in the private cache: 10
Number of blocks in the LLC: 50


In [25]:
def compute_decoded_next_rs(pv, decoded_rs):
    # Generate mask = 11...100...0 with the cross-over from 0 to 1 happening right after the current RS position
    mask = (~decoded_rs + 1) & ~decoded_rs

    # Extract upper and lower portions of PV split right after the current RS position
    upper_pv = pv & mask
    lower_pv = pv & ~mask

    # Find the next set bit position in upperPV
    decoded_next_rs_upper = upper_pv & (~upper_pv + 1)

    # Find the next set bit position in lowerPV
    decoded_next_rs_lower = lower_pv & (~lower_pv + 1)

    # Compute the final output
    if decoded_next_rs_upper == 0:
        decoded_next_rs = decoded_next_rs_lower
    else:
        decoded_next_rs = decoded_next_rs_upper

    return decoded_next_rs

# Example usage
pv = 0b101010101
decoded_rs = 0b1000
decoded_next_rs = compute_decoded_next_rs(pv, decoded_rs)
print(f"decoded_next_rs: {decoded_next_rs:b}")


decoded_next_rs: 10000


In [27]:
import random

class CacheBlock:
    def __init__(self, tag):
        self.tag = tag
        self.timestamp = 0



In [28]:

class CacheSet:
    def __init__(self, associativity):
        self.associativity = associativity
        self.blocks = []

    def access(self, tag):
        for block in self.blocks:
            if block.tag == tag:
                return True
        return False

    def add_block(self, tag, timestamp):
        if len(self.blocks) < self.associativity:
            self.blocks.append(CacheBlock(tag))
            self.update_timestamp(tag, timestamp)
        else:
            self.evict()
            self.add_block(tag, timestamp)

    def update_timestamp(self, tag, timestamp):
        for block in self.blocks:
            if block.tag == tag:
                block.timestamp = timestamp
                break

    def evict(self):
        lru_block = min(self.blocks, key=lambda x: x.timestamp)
        self.blocks.remove(lru_block)


In [29]:

class Cache:
    def __init__(self, size, associativity, block_size):
        self.size = size
        self.associativity = associativity
        self.block_size = block_size
        self.sets_count = size // (associativity * block_size)
        self.sets = [CacheSet(associativity) for _ in range(self.sets_count)]
        self.access_count = 0

    def access(self, address):
        tag, index = self.parse_address(address)
        if self.is_hit(tag, index):
            self.sets[index].update_timestamp(tag, self.access_count)
        else:
            self.sets[index].add_block(tag, self.access_count)
        self.access_count += 1

    def is_hit(self, tag, index):
        return self.sets[index].access(tag)

    def parse_address(self, address):
        index = (address // self.block_size) % self.sets_count
        tag = address // (self.block_size * self.sets_count)
        return tag, index

def generate_addresses(num_addresses):
    return [random.randint(0, 65535) for _ in range(num_addresses)]


In [30]:

def main():
    cache_size = 32 * 1024
    associativity = 4
    block_size = 64
    num_accesses = 10000

    cache = Cache(cache_size, associativity, block_size)
    addresses = generate_addresses(num_accesses)

    for address in addresses:
        cache.access(address)

    print(f"Simulation finished after {num_accesses} accesses.")

if __name__ == "__main__":
    main()

Simulation finished after 10000 accesses.


**Final** Run

In [31]:
import random



class Cache:
    def __init__(self, size, associativity, block_size, ziv_enabled=False):
        self.size = size
        self.associativity = associativity
        self.block_size = block_size
        self.sets_count = size // (associativity * block_size)
        self.sets = [CacheSet(associativity) for _ in range(self.sets_count)]
        self.access_count = 0
        self.miss_count = 0
        self.ziv_enabled = ziv_enabled

    def access(self, address):
        tag, index = self.parse_address(address)
        if self.is_hit(tag, index):
            self.sets[index].update_timestamp(tag, self.access_count)
        else:
            self.miss_count += 1
            if self.ziv_enabled:
                self.evict_and_relocate(tag, index)
            else:
                self.sets[index].add_block(tag, self.access_count)
        self.access_count += 1

   
    def is_hit(self, tag, index):
        return self.sets[index].access(tag)

    def parse_address(self, address):
        index = (address // self.block_size) % self.sets_count
        tag = address // (self.block_size * self.sets_count)
        return tag, index
    def evict_and_relocate(self, tag, index):
        # Implement ZIV block relocation protocol
        pass



In [32]:

def main():
    cache_size = 32 * 1024
    associativity = 4
    block_size = 64
    num_accesses = 10000

    # Without ZIV LLC design (baseline LLC policy)
    cache_without_ziv = Cache(cache_size, associativity, block_size)
    # With ZIV LLC design (block relocation protocol)
    cache_with_ziv = Cache(cache_size, associativity, block_size, ziv_enabled=True)

    addresses = generate_addresses(num_accesses)

    for address in addresses:
        cache_without_ziv.access(address)
        cache_with_ziv.access(address)

    miss_rate_without_ziv = cache_without_ziv.miss_count / num_accesses
    miss_rate_with_ziv = cache_with_ziv.miss_count / num_accesses

    print(f"Miss rate without ZIV LLC design: {miss_rate_without_ziv:.2%}")
    print(f"Miss rate with ZIV LLC design: {miss_rate_with_ziv:.2%}")

if __name__ == "__main__":
    main()

Miss rate without ZIV LLC design: 52.35%
Miss rate with ZIV LLC design: 100.00%
